In [3]:
import torch

### 3.
Hàm softmax dùng exponential function (e) mà không phải số khác như 2, 3 hoặc 4:
do $e^x$ đạo hàm được và có đạo hàm đẹp hơn các số tự nhiên ($=e^x$)

### 1.
Cho bài toán XOR,

Implement thuật toán neural network để giải bài toán


In [4]:
import torch.nn as nn
from torch.autograd import Variable

In [5]:
# create data
X = torch.Tensor([[0., 0.],
               [0., 1.],
               [1., 0.],
               [1., 1.]])

y = torch.Tensor([0., 1., 1., 0.]).reshape(X.shape[0], 1)

In [6]:
class XOR(nn.Module):
    def __init__(self):
        super(XOR, self).__init__()
        self.linear = nn.Linear(2, 2)
        self.Sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(2, 1)

    def forward(self, input):
      x = self.linear(input)
      sig = self.Sigmoid(x)
      yh = self.linear2(sig)
      return yh

In [7]:
xor_network = XOR()

In [17]:
epochs = 1000
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(xor_network.parameters(), lr = 0.03)
all_losses = []
current_loss = 0
plot_every = 50

for epoch in range(epochs):

  # input training example and return the prediction
  yhat = xor_network.forward(X)

  # calculate MSE loss
  loss = mseloss(yhat, y)

  # backpropogate through the loss gradiants
  loss.backward()

  # update model weights
  optimizer.step()

  # remove current gradients for next iteration
  optimizer.zero_grad()

  # append to loss
  current_loss += loss
  if epoch % plot_every == 0:
      all_losses.append(current_loss / plot_every)
      current_loss = 0


In [21]:
# show weights and bias
for name, param in xor_network.named_parameters():
    if param.requires_grad:
        print(name, param.data)

linear.weight tensor([[-5.2789, -5.2308],
        [-2.7595, -2.7551]])
linear.bias tensor([1.3233, 3.4728])
linear2.weight tensor([[-2.0202,  1.8657]])
linear2.bias tensor([-0.2141])


In [24]:
# test input
input = torch.tensor([1., 1.])
out = xor_network(input)
print(out.round())

input = torch.tensor([0., 0.])
out = xor_network(input)
print(out.round())

tensor([0.], grad_fn=<RoundBackward0>)
tensor([0.], grad_fn=<RoundBackward0>)


### 2. Dataset MNIST
- Implement thuật toán neural network với dữ liệu trên sử dụng Pytorch
- Chỉnh learning rate, vẽ đồ thị loss tương ứng.
- Chia tập train/test tỉ lệ 80/20, tính các chỉ số accuracy, recall, f1-score trên tập test.